<a href="https://colab.research.google.com/github/ngdodd/transformers/blob/master/ngdodd_CSE576_Quail_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#_________________________
# Finetune Pretrained Models for QuAIL

QuAIL: http://text-machine.cs.uml.edu/lab2/projects/quail/

Install forked HF transformers repo :)

In [ ]:
!rm -rf transformers/
!git clone https://github.com/ngdodd/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Cloning into 'transformers'...
remote: Enumerating objects: 49872, done.
remote: Total 49872 (delta 0), reused 0 (delta 0), pack-reused 49872
Receiving objects: 100% (49872/49872), 129.43 MiB | 22.83 MiB/s, done.
Resolving deltas: 100% (34768/34768), done.
Checking out files: 100% (1540/1540), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 2.9MB 50.6MB/s 
     |████████████████████████████████| 890kB 54.5MB/s 
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1297093 sha256=ad55a8255000cbdbe73183b14ecb032b78f980cdfc60ff881b7544280b1cd92b
  Stored in directory: /tmp/pip-ephem-wheel-cache-pv54fvew/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremo

     |████████████████████████████████| 17.7MB 219kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
/content


Mount google drive for direct file access if desired.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Download project data:

In [ ]:
!mkdir data
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/train.jsonl -O data/train.jsonl
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/dev.jsonl -O data/dev.jsonl
!wget https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/challenge.jsonl -O data/challenge.jsonl
!wget https://raw.githubusercontent.com/ngdodd/transformers/master/examples/multiple-choice/datasets/synthetic/with_sequential/synthetic_with_nei_cleaned_dev.jsonl -O data/synthetic_dev.jsonl
!wget https://raw.githubusercontent.com/ngdodd/transformers/master/examples/multiple-choice/datasets/synthetic/with_sequential/synthetic_with_nei_cleaned_train.jsonl -O data/synthetic_train.jsonl
!wget https://raw.githubusercontent.com/ngdodd/transformers/master/examples/multiple-choice/datasets/synthetic/without_sequential/synthetic_with_nei_without_sequential_cleaned_dev.jsonl -O data/synthetic_without_sequential_dev.jsonl
!wget https://raw.githubusercontent.com/ngdodd/transformers/master/examples/multiple-choice/datasets/synthetic/without_sequential/synthetic_with_nei_wihtout_sequential_cleaned_train.jsonl -O data/synthetic_without_sequential_train.jsonl
!wget https://raw.githubusercontent.com/ngdodd/transformers/60ac3314e2059cad0fa2e7f7c3d1944b870a238f/examples/multiple-choice/datasets/synthetic/with_quail/with_sequential/synthetic_with_quail_train.jsonl -O data/synthetic_with_quail_train.jsonl
!wget https://raw.githubusercontent.com/ngdodd/transformers/master/examples/multiple-choice/datasets/synthetic/with_quail/with_sequential/synthetic_with_quail_dev.jsonl -O data/synthetic_with_quail_dev.jsonl
!wget https://raw.githubusercontent.com/ngdodd/transformers/60ac3314e2059cad0fa2e7f7c3d1944b870a238f/examples/multiple-choice/datasets/synthetic/with_quail/without_sequential/synthetic_without_seq_with_quail_train.jsonl -O data/synthetic_without_seq_with_quail_train.jsonl
!wget https://raw.githubusercontent.com/ngdodd/transformers/60ac3314e2059cad0fa2e7f7c3d1944b870a238f/examples/multiple-choice/datasets/synthetic/with_quail/without_sequential/synthetic_without_seq_with_quail_dev.jsonl -O data/synthetic_without_seq_with_quail_dev.jsonl
!wget https://raw.githubusercontent.com/ngdodd/transformers/master/examples/multiple-choice/datasets/synthetic/quail_dev_key.json -O data/quail_dev_key.json
!wget https://raw.githubusercontent.com/ngdodd/transformers/master/examples/multiple-choice/datasets/synthetic/synthetic_dev_key.json -O data/synthetic_dev_key.json
!wget https://raw.githubusercontent.com/ngdodd/transformers/master/examples/multiple-choice/datasets/synthetic/synthetic_with_quail_dev_key.json -O data/synthetic_with_quail_dev_key.json

--2020-11-30 10:07:30--  https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24941234 (24M) [text/plain]
Saving to: ‘data/train.jsonl’

data/train.jsonl    100%[===================>]  23.79M  --.-KB/s    in 0.1s    

2020-11-30 10:07:32 (196 MB/s) - ‘data/train.jsonl’ saved [24941234/24941234]

--2020-11-30 10:07:32--  https://raw.githubusercontent.com/text-machine-lab/quail/master/quail_v1.3/json/dev.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

Initialize wandb to track and monitor the training session:

In [ ]:
!pip install wandb
import wandb
wandb.init()

     |████████████████████████████████| 1.8MB 14.8MB/s 
     |████████████████████████████████| 133kB 50.0MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=dad12c840458c592252eac2c0da4102ed8389ad1ac569306fe45304b80ef1f59
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built subprocess32


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#_________________________

#Training

**Baseline Bert** (from QuAIL paper):

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path bert-base-uncased \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 400 \
  --output_dir quail_out/bert_base_uncased_400 \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

**RoBERTa-Base**

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path /content/quail_out/roberta_base \
  --do_train \
  --do_eval \
  --data_dir ./data \
  --learning_rate 1e-5 \
  --num_train_epochs 1 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=2 \
  --per_device_train_batch_size=2 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-base-reasoning_classification_emulation-2 \
  --logging_steps 25 \
  --quail_dev_key_path ./data/quail_dev_key.json \
  --custom_dev_key_path ./data/quail_dev_key.json \
  --eval_on_training_set \

2020-11-30 05:25:07.020188: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 05:25:09 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/30/2020 05:25:09 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='quail_out/roberta_base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=25, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_05-25-09_dc39da6ca396', log

**RoBERTa-Large**

In [ ]:
 !python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-large \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_large \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

**Longformer**:

Install longformer:

In [ ]:
pip install git+https://github.com/allenai/longformer.git

Train:

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path allenai/longformer-base-4096 \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 650 \
  --output_dir quail_out/longformer_base_4096 \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

**RoBERTA** (using Quail dataset originally in the Swag dataset format)

Note: the only thing different here is that the source dataset files used are in the Swag dataset format. Prior to the training loop, the script will parse/format, tokenize, and then cache the entire train and dev datasets into a common data structure that is used across all of the data Processors. This was done originally due to there not being a data Processor available for QuAIL in huggingface/transformers out of the box. The prior RoBERTa training block above uses the newly introduced QuAIL data Processor, and results for the two should be equivalent.

Reformat Quail data into the Swag format for use with the multiple choice script's Swag data processor:

In [ ]:
import pandas as pd
from datasets import load_dataset

# Load and process the quail train, val, and test datasets
quail = load_dataset('quail').map(lambda e: e.update({'a':e['answers'][0], 'b':e['answers'][1], 'c':e['answers'][2], 'd':e['answers'][3]}) or e)

# Convert and write quail data to swag formatted csv files
reindex = ['context_id', 'id', 'domain', 'context', 'question', 'question_id', 'a', 'b', 'c', 'd', 'correct_answer_id']
pd.DataFrame(list(quail['train'])).reindex(columns=reindex).to_csv('quail/train.csv')
pd.DataFrame(list(quail['validation'])).reindex(columns=reindex).to_csv('quail/val.csv')
pd.DataFrame(list(quail['challenge'])).reindex(columns=reindex).to_csv('quail/test.csv')

Train:

In [ ]:
 !python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name swag \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base_swag \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

#Results Summary

Model Name | Eval Accuracy
--- | ---
Baseline BERT | 0.56
Roberta | 0.65
Longformer | 0.68

#Inference

Test with pre-trained model (no-finetuning)

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

#Experimental

#### Deberta for MCQ

Delete any current transformers repo and clone+install the deberta mcq branch:

In [ ]:
!rm -rf transformers
!git clone -b deberta-for-multiple-choice https://github.com/ngdodd/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Train:

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path microsoft/deberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/deberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output

#### DeBERTa MCQ with Reasoning Task Labels

Delete any current transformers repo and clone+install the mcq-with-reasoning-type branch:

In [ ]:
!rm -rf transformers
!git clone -b mcq-with-reasoning-type https://github.com/ngdodd/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Train:

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path microsoft/deberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/deberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --with_reasoning_types

#### RoBERTa-Base MCQ with Reasoning Task Labels

Delete any current transformers repo and clone+install the roberta-mcq-with-reasoning-types branch:

In [ ]:
!rm -rf transformers
!git clone -b roberta-mcq-with-reasoning-types https://github.com/ngdodd/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Train:

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./quail \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-swag+hellaswag+cosmos_qa \
  --logging_steps 50

## Upstream Reasoning Classification --> Downstream QA Model

In [ ]:
!rm -rf transformers/
!git clone https://github.com/ngdodd/transformers.git
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
!pip install pyarrow==2.0.0
%cd ..

Emulate perfect reasoning type classifier by prepending reasoning type to each question. This will be used to validate the approach is useful for the downstream QA task in the best possible scenario (perfect reasoning type classification). If the downstream QA task doesn't perform well on this, then the idea isn't worth pursuing further.

In [ ]:
import json

with open('./data/train.jsonl', 'r') as f, open('train.jsonl', 'w') as w:
  for line in f:
    data = json.loads(line)
    data['question'] = "Question type: {}. {}".format(data['question_type'].replace('_', ' '), data['question'])
    w.write(json.dumps(data) + '\n')

with open('./data/dev.jsonl', 'r') as f, open('dev.jsonl', 'w') as w:
  for line in f:
    data = json.loads(line)
    data['question'] = "Question type: {}. {}".format(data['question_type'].replace('_', ' '), data['question'])
    w.write(json.dumps(data) + '\n')

Paste the following as a new class in utils_multiple_choice.py and create a new entry corresponding to it in processors and MULTIPLE_CHOICE_TASKS

In [ ]:
"""
class QuailReasoningProcessor(QuailProcessor):
    def __init__(self):
        self.reasoning_type_dict = {'Character_identity': 0, 'Causality': 1, 'Event_duration': 2,
                                    'Subsequent_state': 3, 'Factual': 4, 'Belief_states': 5,
                                    'Entity_properties': 6, 'Unanswerable':7, 'Temporal_order':8}
    
    def get_labels(self):
        """See base class."""
        return list(self.reasoning_type_dict.keys())
    
    def _create_examples(self, lines, type):
        examples = [
            InputExample(
                example_id=qa_entry["id"],
                question=qa_entry["question"],
                contexts=[qa_entry["context"] for _ in range(len(self.reasoning_type_dict))],
                endings=[list(self.reasoning_type_dict.keys())[k] for k in range(len(self.reasoning_type_dict))],
                label=qa_entry["question_type"],
            )
            for qa_entry in [json.loads(line) for line in lines]
        ]
        return examples
"""

Reinstall package with modified utils_multiple_choice.py file

In [ ]:
%cd transformers/
!pip install .
%cd ..

/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1297093 sha256=c8949a16448009a2e89d68e5cad9b8e48bf362c7f5deaa455aabd690bc833d37
  Stored in directory: /tmp/pip-ephem-wheel-cache-2s_o7ydt/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 3.4.0
    Uninstalling transformers-3.4.0:
      Successfully uninstalled transformers-3.4.0


/content


Create new dataset

In [ ]:
import json
from datasets import load_dataset

synthetic_with_quail_without_seq_train = load_dataset('json', data_files='./data/synthetic_without_seq_with_quail_train.jsonl')['train']
synthetic_with_quail_without_seq_dev = load_dataset('json', data_files='./data/synthetic_without_seq_with_quail_dev.jsonl')['train']

with open('./data/synthetic_without_seq_with_quail_reasoning_train.jsonl', 'w', encoding='utf-8') as w:
    for entry in synthetic_with_quail_without_seq_train:
        entry['correct_answer_id'] = entry['question_type']
        w.write(json.dumps(entry) + '\n')

with open('./data/synthetic_without_seq_with_quail_reasoning_dev.jsonl', 'w', encoding='utf-8') as w:
    for entry in synthetic_with_quail_without_seq_dev:
        entry['correct_answer_id'] = entry['question_type']
        w.write(json.dumps(entry) + '\n')

Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-75d56dec875ca6ab/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)
Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-e6bf455afbabcf80/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)


Train the upstream reasoning classifier

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail-reasoning \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./data \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=1 \
  --per_device_train_batch_size=1 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-base-reasoning_classification \
  --logging_steps 25 \
  --custom_training_filename synthetic_without_seq_with_quail_reasoning_train.jsonl \
  --custom_eval_filename synthetic_without_seq_with_quail_reasoning_dev.jsonl \
  --eval_on_training_set \
  --overwrite_cache

2020-11-30 10:51:59.949551: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 10:52:01 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/30/2020 10:52:01 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='quail_out/roberta_base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=25, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_10-52-01_650e5d6ad2e4', log

In [ ]:
from google.colab import files
#For the folder you have to zip it first and can only download later on
!zip -r reasoning_classifier_results.zip quail_out
#Download files
files.download('reasoning_classifier_results.zip')

  adding: quail_out/ (stored 0%)
  adding: quail_out/roberta_base/ (stored 0%)
  adding: quail_out/roberta_base/training_args.bin (deflated 44%)
  adding: quail_out/roberta_base/checkpoint-1500/ (stored 0%)
  adding: quail_out/roberta_base/checkpoint-1500/training_args.bin (deflated 44%)
  adding: quail_out/roberta_base/checkpoint-1500/scheduler.pt (deflated 49%)
  adding: quail_out/roberta_base/checkpoint-1500/trainer_state.json (deflated 76%)
  adding: quail_out/roberta_base/checkpoint-1500/pytorch_model.bin (deflated 10%)
  adding: quail_out/roberta_base/checkpoint-1500/config.json (deflated 56%)
  adding: quail_out/roberta_base/checkpoint-1500/optimizer.pt (deflated 19%)
  adding: quail_out/roberta_base/Train_Eval_metrics.json (deflated 15%)
  adding: quail_out/roberta_base/checkpoint-2500/ (stored 0%)
  adding: quail_out/roberta_base/checkpoint-2500/training_args.bin (deflated 44%)
  adding: quail_out/roberta_base/checkpoint-2500/scheduler.pt (deflated 49%)
  adding: quail_out/rob

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Use the reasoning classifier output to annotate the QA dataset

In [ ]:
!mkdir annotated_data
!pip install datasets

In [ ]:
from datasets import load_dataset

def load_dict(path):
  with open(path, 'r', encoding='utf-8') as f:
      return json.load(f)

# Load the predictions from the trained upstream reasoning classifier on the various datasets
custom_dev_preds = load_dict('./quail_out/roberta_base/Custom_Dev_Eval_preds.json')
quail_dev_preds = load_dict('./quail_out/roberta_base/Quail_Dev_Eval_preds.json')
custom_train_preds = load_dict('./quail_out/roberta_base/Train_Eval_preds.json')
type_names = ['Character_identity', 'Causality', 'Event_duration', 'Subsequent_state','Factual', 'Belief_states', 'Entity_properties', 'Unanswerable', 'Temporal_order']

# Annotate the synthetic training set
synthetic_without_seq_with_quail_train = load_dataset('json', data_files='./data/synthetic_without_seq_with_quail_train.jsonl')['train']
with open('./annotated_data/synthetic_without_seq_with_quail_with_reasoning_train.jsonl', 'w', encoding='utf-8') as w:
    for entry in synthetic_without_seq_with_quail_train:
        reasoning_type = type_names[custom_train_preds[entry['id']]]
        entry['question'] = "Question type: {}. {}".format(reasoning_type.replace('_', ' '), entry['question'])
        w.write(json.dumps(entry) + '\n')

# Annotate the synthetic dev set
synthetic_without_seq_with_quail_dev = load_dataset('json', data_files='./data/synthetic_without_seq_with_quail_dev.jsonl')['train']
with open('./annotated_data/synthetic_without_seq_with_quail_with_reasoning_dev.jsonl', 'w', encoding='utf-8') as w:
    for entry in synthetic_without_seq_with_quail_dev:
        reasoning_type = type_names[custom_dev_preds[entry['id']]]
        entry['question'] = "Question type: {}. {}".format(reasoning_type.replace('_', ' '), entry['question'])
        w.write(json.dumps(entry) + '\n')

# Annotate the quail dev set
quail_dev = load_dataset('json', data_files='./data/dev.jsonl')['train']
with open('./annotated_data/quail_with_reasoning_dev.jsonl', 'w', encoding='utf-8') as w:
    for entry in quail_dev:
        reasoning_type = type_names[quail_dev_preds[entry['id']]]
        entry['question'] = "Question type: {}. {}".format(reasoning_type.replace('_', ' '), entry['question'])
        w.write(json.dumps(entry) + '\n')

Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-75d56dec875ca6ab/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)
Using custom data configuration default
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-e6bf455afbabcf80/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514)
Using custom data configuration default


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ef4fb0dccfb36032/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514. Subsequent calls will reuse this data.


Copy annotated data sets to default quail dev location to be used automatically during the run.

In [ ]:
!cp ./annotated_data/quail_with_reasoning_dev.jsonl data/dev.jsonl
!cp ./annotated_data/synthetic_without_seq_with_quail_with_reasoning_dev.jsonl ./data/synthetic_without_seq_with_quail_with_reasoning_dev.jsonl
!cp ./annotated_data/synthetic_without_seq_with_quail_with_reasoning_train.jsonl ./data/synthetic_without_seq_with_quail_with_reasoning_train.jsonl

Train a new QA model with the annotated datasets

In [ ]:
!python ./transformers/examples/multiple-choice/run_multiple_choice.py \
  --task_name quail \
  --model_name_or_path roberta-base \
  --do_train \
  --do_eval \
  --data_dir ./data \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --max_seq_length 512 \
  --output_dir quail_out/roberta_base \
  --per_device_eval_batch_size=2 \
  --per_device_train_batch_size=2 \
  --gradient_accumulation_steps 25 \
  --overwrite_output \
  --cache_dir colab_cache \
  --run_name roberta-base--with_upsteam_reasoning_classification \
  --logging_steps 25 \
  --quail_dev_key_path ./data/quail_dev_key.json \
  --custom_training_filename synthetic_without_seq_with_quail_with_reasoning_train.jsonl \
  --custom_eval_filename synthetic_without_seq_with_quail_with_reasoning_dev.jsonl \
  --custom_dev_key_path ./data/synthetic_with_quail_dev_key.json \
  --eval_on_training_set \
  --overwrite_cache

2020-11-30 17:42:05.689503: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 17:42:07 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/30/2020 17:42:07 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='quail_out/roberta_base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=25, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_17-42-07_650e5d6ad2e4', log